In [ ]:
using ITensors, TensorKit, TensorOperations

In [ ]:
function potential(t, U, μ, L, lobe)

    sites = siteinds("Qudit",L; dim=5,  conserve_qns=true, conserve_number=true)

    H = OpSum()
    for i in (1:L-1)
        H += -μ - U/2, "n", i
        H += U/2, "n", i, "n", i
        H += -t, "a", i, "adag", i+1
        H += -t, "adag", i, "a", i+1
    end
    # periodic boundary conditions.
    H += -t, "a", 1, "adag", L
    H += -t, "adag", 1, "a", L

    Ham = MPO(H, sites)
    
    lb = string(lobe)

    state = [isodd(n) ? lb : lb for n=1:L]
    state_p = [isodd(n) ? lb : lb for n=1:L]
    state_p[1] = string(lobe+1)

    state_m = [isodd(n) ? lb : lb for n=1:L]
    state_m[1] = string(lobe-1)

    psi0 = randomMPS(sites,state)
    psi0_p = randomMPS(sites,state_p)
    psi0_m = randomMPS(sites,state_m)
  
    nsweeps = 10
    maxdim = [40, 80, 400, 400, 800, 800, 1000, 1000, 1000, 1000]
    cutoff = [1e-12]
  
    energy, psi = dmrg(Ham,psi0; nsweeps, maxdim, cutoff, outputlevel=0)
    N = expect(psi, "n")
    println("GS:")
    println(N)
    println(sum(N))

    energy_p, psi_p = dmrg(Ham,psi0_p; nsweeps, maxdim, cutoff, outputlevel=0)
    N = expect(psi_p, "n")
    println("+1:")
    println(N)
    println(sum(N))

    energy_m, psi_m = dmrg(Ham,psi0_m; nsweeps, maxdim, cutoff, outputlevel=0)
    N = expect(psi_m, "n")
    println("-1:")
    println(N)
    println(sum(N))

    return energy_p - energy, energy-energy_m
end

In [ ]:
potential(0.1, 2, 0, 12, 1)

In [ ]:
potential(0.1, 2, 0, 12, 1)

In [ ]:
U_val = 2
μ_val = 0
sites = 128
lobe = 1

upper = zeros(0)
lower = zeros(0)

In [34]:
for t in range(0.0, 2.0, step=0.1)
    up, low = potential(t, U_val, μ_val, sites, lobe)
    append!(upper, up)
    append!(lower, low)
end

In [ ]:
println(upper)
println(lower)